# PEBL vs k-means



In [ ]:
import numpy as np
import os
from matplotlib import pyplot as plt
import sys
sys.path.insert(0,'./DATA')
from MeshData import fluid_mesh_data
from sklearn.cluster import KMeans
%matplotlib inline 


In [ ]:
#binary tree for PEBL clustering

class Node:
    def __init__(self, val):
        self.left = None
        self.right = None
        self.val = val

    def leaves(self):
        current_nodes = [self]
        leaves = []

        while len(current_nodes) > 0:
            next_nodes = []
            for node in current_nodes:
                if node.left is None and node.right is None:
                    leaves.append(node)
                    continue
                if node.left is not None:
                    next_nodes.append(node.left)
                if node.right is not None:
                    next_nodes.append(node.right)
            current_nodes = next_nodes
        return leaves


#other functions
def E_p(u, c):
    """
    c: direction vector onto which to project
    u: vector or colection of column vectors to project onto the direction of c
    """
    c = c.reshape(-1,1)
    if len(u.shape)==1:
        u = u.reshape(-1,1)
    projection_of_u_onto_c = ((c@c.T) / (c.T@c)) @ u
    projection_error = np.linalg.norm(u - projection_of_u_onto_c, axis=0) / np.linalg.norm(u, axis=0)
    return projection_error

def PEBL(Snapshots, bisection_tolerance=0.15,  POD_tolerance=1e-3):
    #stage 1, generation of bisection tree with accuracy 'bisection_tolerance'
    max_index = np.argmax( np.linalg.norm(Snapshots, axis=0) )
    first_snapshot = Snapshots[:,max_index]
    Tree = Node([first_snapshot, Snapshots])
    bisect_flag = True
    while bisect_flag == True:
        bisect_flag = False
        for leave in Tree.leaves():
            errors = E_p(leave.val[1], leave.val[0])
            max_error = max(errors)
            #print(max_error)
            if max_error > bisection_tolerance:
                bisect_flag = True
                #find next anchor point
                max_index = np.argmax(errors)
                c_new = leave.val[1][:,max_index]
                new_errors = E_p(leave.val[1], c_new)
                indexes_left = np.where( errors <= new_errors)
                indexes_right = np.where( errors > new_errors)
                #divide the snapshots among the two children
                leave.left =  Node([leave.val[0], leave.val[1][:,indexes_left[0]]])
                leave.right = Node([c_new, leave.val[1][:,indexes_right[0]]])
                leave.val[1] = None
    #stage 2, generation of local POD bases'
    for leave in Tree.leaves():
        Phi_i = ObtainBasis(leave.val[1], POD_tolerance)
        leave.val.append(Phi_i)

    return Tree

def ObtainBasis(Snapshots, truncation_tolerance=0):
        U,_,_= truncated_svd(Snapshots,truncation_tolerance)
        return U

def truncated_svd(Matrix, epsilon=0):

    M,N=np.shape(Matrix)
    dimMATRIX = max(M,N)
    U, s, V = np.linalg.svd(Matrix, full_matrices=False) #U --> M xN, V --> N x N
    V = V.T
    tol = dimMATRIX*np.finfo(float).eps*max(s)/2
    R = np.sum(s > tol)  # Definition of numerical rank
    if epsilon == 0:
        K = R
    else:
        SingVsq = np.multiply(s,s)
        SingVsq.sort()
        normEf2 = np.sqrt(np.cumsum(SingVsq))
        epsilon = epsilon*normEf2[-1] #relative tolerance
        T = (sum(normEf2<epsilon))
        K = len(s)-T
    K = min(R,K)
    return U[:, :K], s[:K], V[:, :K]



In [ ]:
# functions to test k-means and PEBL on 2D data

def kmeans_test(test_data):
    plt.figure(figsize=(12, 12))
    # Incorrect number of clusters
    kmeans_object = KMeans(n_clusters=5).fit(test_data)
    plt.scatter(test_data[:, 0], test_data[:, 1], c=kmeans_object.labels_)
    centroids_to_plot = (kmeans_object.cluster_centers_).T
    plt.scatter(centroids_to_plot[0,:], centroids_to_plot[1,:], c='r', s= 50)
    plt.title("k means clustering")
    
def pebl_test(test_data):
    Tree = PEBL(test_data.T, 0.68)
    plt.figure(figsize=(12, 12))
    #print(len(Tree.leaves()))
    for leaf in Tree.leaves():
        plt.scatter(leaf.val[1][0,:], leaf.val[1][1,:])
        plt.scatter(leaf.val[0][0], leaf.val[0][1], c='k', s= 150)
    plt.title('PEBL clustering')

In [ ]:
# test case with random data from PEBL from Amsallem

test_data = np.random.rand(1000,2)
test_data -= test_data.mean(axis = 0) #centering
kmeans_test(test_data)
pebl_test(test_data)

# Clustering real data

In [ ]:
#Functions for plotting results
nodes, eles = fluid_mesh_data()
x, y = nodes.T

def PlotFluidSnapshot(snapshot):
    plt.figure(figsize=(10,10))
    plt.tricontourf(x, y, eles, snapshot)
    plt.gca().set_aspect('equal', adjustable='box')

    


## 2D CFD Example

![title](./DATA/2DFluid.gif)

In [ ]:
#loading fluid mesh

nodes, eles = fluid_mesh_data()
print('shape of elements array: ', np.shape(eles))
print('shape of nodes array: ',np.shape(nodes))

In [ ]:
#loading snapshots matrix of 2D CFD example

SnapshotsFluid = np.load('./DATA/snapshot_matrix.npy')
print(f'The shape of the fluid snapshots matrix is: {SnapshotsFluid.shape}')

In [ ]:
#spliting data into x-velocity, y-velocity and pressure

total_number_of_rows = SnapshotsFluid.shape[0]
vx = SnapshotsFluid[0:total_number_of_rows:3,:]# velocity x
vy = SnapshotsFluid[1:total_number_of_rows:3,:]# velocity y
p = SnapshotsFluid[2:total_number_of_rows:3,:]# pressure

print('x velocity nodal data size: ', np.shape(vx))
print('y velocity nodal data size: ',np.shape(vy))
print('pressure nodal data size: ',np.shape(p))


In [ ]:
#visualizing the data

norm_of_velocity = np.sqrt(np.power(vx,2) + np.power(vy,2))
snapshot_to_print = 250

PlotFluidSnapshot(norm_of_velocity[:,snapshot_to_print])
PlotFluidSnapshot(p[:,snapshot_to_print])


In [ ]:
#create the clustering ignoring the first 150/400 snapshots

Tree = PEBL(norm_of_velocity[:,150:],bisection_tolerance=0.1 )

In [ ]:
list_of_leaves = [leave.val for leave in Tree.leaves()]
print("number of clusters created: ",len(list_of_leaves))


In [ ]:
#observe the snapshots on the leaves

for i in range(len(list_of_leaves)):
    PlotFluidSnapshot(list_of_leaves[i][0])

In [ ]:
#different clustering

Tree = PEBL(norm_of_velocity,bisection_tolerance=0.1 )

list_of_leaves = [leave.val for leave in Tree.leaves()]
print("number of clusters created: ",len(list_of_leaves))

#observe the snapshots on the leaves

for i in range(len(list_of_leaves)):
    PlotFluidSnapshot(list_of_leaves[i][0])
